In [0]:
import torch
def test(points, rot_mat, mask) :
  
  size = (128, 128) # N x C x H x W apparently
  base = test_compare()
  base = base.expand((1,1,size[0],size[1]))
  learning_rate = 0.1
  #x_rot = torch.tensor([0.0], dtype=torch.float32, requires_grad=True)
  #y_rot = torch.tensor([0.0], dtype=torch.float32, requires_grad=True)
  #z_rot = torch.tensor([0.0], dtype=torch.float32, requires_grad=True)
  
  ndc_mat = gen_ndc(size)
  #rot_mat = gen_rot(x_rot, y_rot, z_rot)
  trans_mat = gen_trans(0.0, 0.0, 2.0)
  proj_mat = gen_perspective(math.radians(90), 1.0, 1.0, 10.0)
  rot_mat.retain_grad() # Need this because it's not a leaf node?!?!

  for i in range(200):
    
    model_mat = torch.matmul(trans_mat, rot_mat)
    o = torch.matmul(model_mat, points)
    q = torch.matmul(proj_mat, o)
    # Divide through by W seems to work with a handy mask and sum
    w = q * mask
    w = torch.sum(w,1,keepdim=True)
    r = q / w
  
    s = r.narrow(1, 0, 2).reshape(1,1,-1,2)
    output = F.grid_sample(base, s)
    gauss_point = torch.tensor([[[[1.0]]]], requires_grad=True,\
      dtype=torch.float32)

    criterion = torch.nn.MSELoss()
    loss = criterion(output, gauss_point)
    loss.backward(retain_graph=True)

    with torch.no_grad():
      rot_mat -= learning_rate * rot_mat.grad
      #print("rot_mat", rot_mat)
      #loss.backward(retain_graph=True)
      #print("loss", rot_mat._grad)
      #tn = grad(loss,rot_mat)
      rot_mat.grad.zero_()
      splatted = splat(points, model_mat, proj_mat, ndc_mat, size)
      img = Image.fromarray(np.uint8(splatted.detach().numpy() * 255))
      img.save("torch" + str(i).zfill(3) + ".jpg", "JPEG")
  return rot_mat
